In [26]:
using Revise
"." in LOAD_PATH || push!(LOAD_PATH, ".")

true

In [27]:
using PATHSolver
using TraceCalls
using JuMP
using Base.Test
import PATHInterface
reload("PATHInterface")

In [28]:
# find x such that 0 <= x <= 1
lb = [0.]
ub = [1.]
f = x -> zeros(1)
options(output=:no)
status, z, F = solveLCP(f, lb, ub);
@test z ≈ [0]

Reading options file path.opt
Read of options file complete.

1 row/cols, 0 non-zeros, 0.00% dense.

Path 4.7.03 (Wed May 28 13:11:13 2014)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


Test Passed

In [29]:
# x == 0 or y == 0
lb = [-Inf, -Inf]
ub = [Inf, Inf]
f = z -> [z[2], z[1]]
options(output=:no)
status, z, F = solveLCP(f, lb, ub)

Reading options file path.opt
Read of options file complete.

2 row/cols, 2 non-zeros, 50.00% dense.

Path 4.7.03 (Wed May 28 13:11:13 2014)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


In [30]:
# phi = x - 1
# phi >= 0 \perp lambda >= 0
# z = [x, lambda]
# f = [0, x - 1]
# lb = [-Inf, 0]
# ub = [Inf, Inf]

lb = [-Inf, 0]
ub = [Inf, Inf]
f = z -> [0, z[1] - 1]
options(output=:no)
status, z, F = solveLCP(f, lb, ub)

Reading options file path.opt
Read of options file complete.

2 row/cols, 1 non-zeros, 25.00% dense.

Path 4.7.03 (Wed May 28 13:11:13 2014)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


In [31]:
# phi = x - y
# phi >= 0 \perp lambda >= 0
# 0.5 <= y <= 1
# z = [x, y, lambda]

f = z -> [0., 0., z[1] - z[2]]
lb = [-Inf, 0.5, 0]
ub = [Inf, 1.0, Inf]
options(output=:no)
status, z, F = solveLCP(f, lb, ub)

Reading options file path.opt
Read of options file complete.

3 row/cols, 2 non-zeros, 22.22% dense.

Path 4.7.03 (Wed May 28 13:11:13 2014)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


In [32]:
# phi = x - y
# phi >= 0 \perp lambda >= 0
# 0.5 <= y <= 1
# 2x - y >= 10

# convert into:
# phi = x - y
# phi >= 0 \perp lambda >= 0
# 0.5 <= y <= 1
# s1 >= 0
# -2x + y + s1 == -10

# z = [x, y, s1, s2, lambda]
# f = [0, 0, 0, -2x + y + s1 - 10, x - y]

f = z -> [0., 0., 0., -2z[1] + z[2] + z[3] + 10, z[1] - z[2]]
lb = [-Inf, 0.5, 0., -Inf, 0]
ub = [Inf, 1.0, Inf, Inf, Inf]
options(output=:no)
status, z, F = solveLCP(f, lb, ub)

Reading options file path.opt
Read of options file complete.

5 row/cols, 5 non-zeros, 20.00% dense.

Path 4.7.03 (Wed May 28 13:11:13 2014)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


In [33]:
using JuMP

In [34]:
m = Model(solver=PATHInterface.PATHLCPSolver())
@variable m x
@variable m 0.5 <= y <= 1
@constraint m 2x - y >= 10
m
options(output=:no)
solve(m)
@test getvalue(x) ≈ 5.25
@test getvalue(y) ≈ 0.5

Reading options file path.opt
Read of options file complete.

4 row/cols, 3 non-zeros, 18.75% dense.

Path 4.7.03 (Wed May 28 13:11:13 2014)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


In [45]:
m = Model(solver=PATHInterface.PATHLCPSolver())
@variable m x >= 1
PATHInterface.complementarity!(m, x, x - 2)
options(output=:no)
@test solve(m) == :Optimal
getvalue(x)

Reading options file path.opt
Read of options file complete.

1 row/cols, 1 non-zeros, 100.00% dense.

Path 4.7.03 (Wed May 28 13:11:13 2014)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


2.0

In [44]:
get(m.internalModel.lcp)

PATHInterface.LCP{Float64}([1.0], [Inf], [0.0], 
  [1, 1]  =  1.0)